# Sprint11
## 1次元の畳み込みニューラルネットワークスクラッチ

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
(X_train,y_train), (X_test,y_test)=mnist.load_data()
X_train,X_test=X_train.reshape(-1,784),X_test.reshape(-1,784)
X_train,X_test=X_train.astype(float)/255,X_test.astype(float)/255

In [3]:
enc=OneHotEncoder(handle_unknown='ignore',sparse=False)
y_train=enc.fit_transform(y_train[:,np.newaxis])
y_test=enc.transform(y_test[:,np.newaxis])

In [4]:
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2)

In [5]:
class XavierInitializer:
    """
    XavierInitializerクラス
    ※シグモイド関数やハイパボリックタンジェント関数のとき
    """
    def __init__(self,n_nodes1):
        self.sigma=1/np.sqrt(n_nodes1)
       
    def W(self, filter_size):
        W=self.sigma*np.random.randn(filter_size)
        return W
    
    def B(self):
        B=self.sigma*np.random.randn()
        return B

In [6]:
class AdaGrad:
    """
    最適化手法（AdaGrad）
    ある層の重みやバイアスの更新
    Parameters
    ----------
    layer : 更新前の層のインスタンス
    """
    def __init__(self,lr=0.01):
        self.lr=lr
        self.hW=1
        self.hB=1
        
    def update(self, layer):
        self.hW+=layer.dW
        self.hB+=layer.dB
#         layer.W-=self.lr*(1/np.sqrt(self.hW))*layer.dW
#         layer.B-=self.lr*(1/np.sqrt(self.hB))*layer.dB
         

In [7]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr=0.01):
        self.lr = lr
    def update(self,layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        
        layer.W-=self.lr*layer.dW
        layer.B-=self.lr*layer.dB
         

## 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成

In [17]:
class SimpleConv1d:
    """
    シンプルな１次元畳込み層
    Parameters
    ----------
    w : 
      フィルター
    b : 
      バイアス
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    straide : ストライド
    padding : パディング数
    """
    def __init__(self,w,b,initializer=XavierInitializer,optimizer=AdaGrad,stride=1,padding=0,input_size=4,filter_size=3):
        self.optimizer = optimizer()
        self.n_nodes=input_size
        self.initializer=initializer(self.n_nodes)
        # 初期化
        self.W=w
        self.B=b
        self.stride=stride
        self.padding=padding
        
    
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        self.a=X
        self.n_out=self.N_Out()
        
        #インデックス取得
        self.idx=np.array([np.arange(i,i+len(self.W)) for i in range(self.n_out)])
        A=(self.a[self.idx]*w).sum(axis=1)+b
        
        return A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        
        self.dB=dA.sum(axis=0)
        self.dW=np.dot(dA,self.a[self.idx])

        dZ=np.zeros((len(self.a)))
        for j in range(len(dA)):
            temp=0
            for s in range(len(self.W)):
                if j-s<0 or j-s>n_out-1:
                    pass
            else:
                temp+=dA[j-s]*self.W[s]
            dZ[j]=temp
        
        # 更新
        self = self.optimizer.update(self)
        return dZ
    
    #1次元畳み込み後の出力サイズの計算
    def N_Out(self):
        return int(((self.a.shape[0]+2*self.padding-len(self.W))/self.stride)+1)

In [18]:
#テスト用に変数を仮定
x = np.array([1,2,3,4])#入力
w = np.array([3, 5, 7])#重み=フィルタ
b = np.array([1])#バイアス
n_out=2#出力サイズ

#フォワードプロパゲーション
def forward(x,w,b,n_out):
    #インデックス取得＋２次元配列へ
    idx=np.array([np.arange(i,i+len(w)) for i in range(n_out)])
    #出力作成
    a=(x[idx]*w).sum(axis=1)+b #np.array([35, 50])
    return a,idx

#テスト
a,idx=forward(x,w,b,n_out)
a

array([35, 50])

In [19]:
#テスト用に変数を仮定
delta_b = np.array([30])
delta_w = np.array([50, 80, 110])
delta_x = np.array([30, 110, 170, 140])
delta_a = np.array([10, 20])

In [11]:
#バックワード
def backward(x,w,idx):
    delta_b=delta_a.sum(axis=0) #np.array([30])
    delta_w=np.dot(delta_a,x[idx]) #np.array([50,80,110])
    delta_x=np.zeros((len(x)))
    for j in range(len(delta_x)):
        temp=0
        for s in range(len(w)):
            if j-s<0 or j-s>n_out-1:
                pass
            else:
                temp+=delta_a[j-s]*w[s]
            delta_x[j]=temp
    return delta_b,delta_w,delta_x

#テスト
dB,dW,dZ=backward(x,w,idx)
print("dB:{}\n".format(dB),
     "dW:{}\n".format(dW),
     "dZ:{}\n".format(dZ))

dB:30
 dW:[ 50  80 110]
 dZ:[ 30. 110. 170. 140.]



In [20]:
#クラスでも確認
sc1d=SimpleConv1d(w,b)
sc1d.forward(x)
sc1d.backward(delta_a)

TypeError: Cannot cast ufunc subtract output from dtype('float64') to dtype('int64') with casting rule 'same_kind'

## 【問題2】1次元畳み込み後の出力サイズの計算

In [96]:
def N_Out(X,W,padding,stride):
        return ((X.shape[0]+2*padding-len(W))/stride)+1
    
#テスト
N_Out(x,w,1,1)

4.0

## 【問題3】小さな配列での1次元畳み込み層の実験

In [95]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])
#フォワード
a,idx=forward(x,w,b,n_out)
a

#バックワード
dB,dW,dZ=backward(x,w,idx)
print("dB:{}\n".format(dB),
     "dW:{}\n".format(dW),
     "dZ:{}\n".format(dZ))

dB:30
 dW:[ 50  80 110]
 dZ:[ 30. 110. 170. 140.]



In [126]:
#工夫
a = np.empty((2, 3))
indexes0 = np.array([0, 1, 2]).astype(np.int)
indexes1 = np.array([1, 2, 3]).astype(np.int)

a[0] = x[indexes0]*w # x[indexes0]は([1, 2, 3])である
a[1] = x[indexes1]*w # x[indexes1]は([2, 3, 4])である
a = a.sum(axis=1)

x = np.array([1, 2, 3, 4])
indexes = np.array([[0, 1, 2], [1, 2, 3]]).astype(np.int)
x[indexes]

array([[1, 2, 3],
       [2, 3, 4]])

## 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成

In [13]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）

In [119]:
class Conv1d:
    """
    
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self,w,b,stride=1,padding=0):
        self.W = w
        self.B = b
        self.stride=stride
        self.padding=padding
        self.out_put_size=
        self.
         
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        self.A=X
        #インデックス取得
        self.idx=np.array([np.arange(i,i+len(self.W)) for i in range(self.out_put_size)])
        A=(X[idx]*w).sum(axis=1)+b
        
        return A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        
        self.dB=dA.sum(axis=0)
        self.dW=np.dot(dA,self.A[self.idx])

        dZ=np.zeros((len(self.A)))
        for j in range(len(dA)):
            temp=0
            for j in range(len(self.W)):
                if j-s<0 or j-s>n_out-1:
                    pass
            else:
                temp+=dA[j-s]*self.W[s]
            dZ[j]=temp

        return dZ

In [ ]:
def N_Out(x):
        return int(((x.shape[0]+2*padding-len(self.W))/self.stride)+1)

In [25]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）
n_out_put=



(3,)

In [27]:
class CNN2dim_FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, w, b , optimizer,stride, padding):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        #init = initializer
        #self.W = init.W(num_filter)
        #self.B = init.B(num_bias)
        self.W = w
        self.B = b
        self.stride = stride
        self.padding = padding
    
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        self.A = X
        output_size, chanel_size, filter_size = self.W.shape
        feature_size = self.A.shape[1]

        a = np.zeros([output_size, feature_size - 2])
        for output in range(output_size):
            for j in range(filter_size - 1):
                sig = 0
                for chanel in range(chanel_size):
                    for i in range(filter_size):
                        sig += X[chanel, i+j] * self.W[output, chanel, j]
                a[output, j] = sig + b[output]
                
                
            

        
        return a

    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        self.n_out = N_OUT(self.stride, self.padding, self.W, self.A)
        self.LB = np.sum(dA, axis=1)
        
        output_size, chanel_size, filter_size = self.W.shape
        feature_size = self.A.shape[1]
        
        #LWの計算
        self.LW = np.zeros_like(self.W)
        for output in range(output_size):
            for chanel in range(chanel_size):
                for i in range(filter_size):
                    for j in range(filter_size -1):
                        self.LW[output, chanel, i] += dA[output, j]*self.A[chanel, j+i]

                 
                            
        #dZの計算
        dZ = np.zeros_like(self.A)
        for output in range(output_size):
            for chanel in range(chanel_size):
                for j in range(feature_size):
                    sigma=0
                    for s in range(filter_size):
                        if j - s < 0 or j - s > self.n_out -1:
                            pass
                        else:
                            sigma += dA[output,  j-s] * self.W[output, chanel, s]
                    dZ[chanel, j] += sigma        
        
        # 更新
        self = self.optimizer.update(self)
        return dZ

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 21)

## 【問題8】学習と推定